In [ ]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor

# Configure Chrome WebDriver options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode (no GUI)

url = "https://grocerapp.pk/categories"
driver = webdriver.Chrome(options=chrome_options)

driver.get(url)
time.sleep(10)

# Find the container that holds the category links
category_buttons_container = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "/html/body/div/div/main/div[1]/div[2]/div")))
category_buttons = category_buttons_container.find_elements(By.CSS_SELECTOR, "a")

# Store the links in a list
category_links = [button.get_attribute("href") for button in category_buttons]

max_threads = 8

def scrape_category(category_link):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    products = []
    
    try:
        driver.get(category_link)
#         time.sleep(30)
        
        # Extract category name
        my_list = category_link
        my_list=my_list.split('/')
    
        category_name = my_list[-2]
        print(category_name)
    
        product_elements = driver.find_elements(By.CSS_SELECTOR,".MuiTypography-root.MuiTypography-body1")
        product_quantity_elements = driver.find_elements(By.CSS_SELECTOR,".MuiTypography-root.MuiTypography-caption")
        product_price_elements = driver.find_elements(By.CSS_SELECTOR,".MuiTypography-root.MuiTypography-subtitle2")
    
        # Loop through product elements and extract data
        for name_element, quantity_element, price_element in zip(product_elements, product_quantity_elements, product_price_elements):
            product_name = name_element.text
            product_quantity = quantity_element.text
            product_price = price_element.text
            
            products.append({
                "category": category_name,
                "name": product_name,
                "quantity": product_quantity,
                "price": product_price,
                "date": time.strftime("%Y-%m-%d")
            })
        print(all_products)
        
    except Exception as e:
        print(f"An error occurred while processing category {category_link}: {e}")
    
    finally:
        driver.quit()
    
    return products

# Initialize the thread pool
all_products = []
with ThreadPoolExecutor(max_threads) as executor:
    for products in executor.map(scrape_category, category_links):
        all_products.extend(products)

# Save data to CSV file
csv_file = "Catagories_groccer_data.csv"
with open(csv_file, "w", newline="", encoding="utf-8") as file:
    fieldnames = ["category", "name", "quantity", "price", "date"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(all_products)
